In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

# Load the dataset
data = pd.read_csv('../ADC_fil.csv')


In [2]:
# Encode categorical variables
label_encoders = {}
categorical_cols = ['Aircraft.damage', 'Aircraft.Category', 'Make.Grouped', 'Engine.Type', 'Purpose.of.flight', 'Weather.Condition', 'Broad.phase.of.flight']
for column in categorical_cols:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


In [3]:
# Define features and target
X = data[['Aircraft.damage', 'Aircraft.Category', 'Make.Grouped', 'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight', 'Weather.Condition', 'Broad.phase.of.flight']]
y = data['Injury.Target']

# Train-test split: 80-20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)


In [4]:
# Define base models
base_models = [
    ('gb', GradientBoostingClassifier(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('mlp', MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42))
]

# Define the stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(max_iter=1000, class_weight='balanced'),
    cv=5  # Number of folds for cross-validation
)

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)


C:\Users\sunny\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:46:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sunny\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:47:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sunny\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:47:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\sunny\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:47:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

StackingClassifier(cv=5,
                   estimators=[('gb',
                                GradientBoostingClassifier(random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None,
                                              feature_weights=None, gamma...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None, ...)),
                               ('mlp',
                                MLPClassifier(max_iter=300, random_state=42))],
                   final_estimator=LogisticRegression(class_weight='balanced',
                                                      max_iter=1000))

In [25]:
# Example of new data for prediction
new_data = pd.DataFrame({
    'Aircraft.damage': ['Destroyed'],
    'Aircraft.Category': ['Helicopter'],
    'Make.Grouped': ['Bell Helicopter'],
    'Number.of.Engines': [1],
    'Engine.Type': ['Turbo Shaft'],
    'Purpose.of.flight': ['Special Ops'],
    'Weather.Condition': ['IMC'],
    'Broad.phase.of.flight': ['Landing']
})



In [26]:
# Encode the new data using the same label encoders
for column in categorical_cols:
    if column in new_data.columns:
        new_data[column] = label_encoders[column].transform(new_data[column])


In [27]:
# Make predictions using the trained stacking classifier
predictions = stacking_clf.predict(new_data)
prediction_probabilities = stacking_clf.predict_proba(new_data)


In [28]:
print("Predictions:", predictions)
print("Prediction Probabilities:", prediction_probabilities)

Predictions: [1]
Prediction Probabilities: [[0.2232987 0.7767013]]
